## 型

Juliaの型システムは動的だが、静的型付システムの方式も一部取り入れている

そのため、ある種の値に対しては、型を判定することができる（型推論）

型を省略した時は値がどんな型であっても許されるが、型注釈を加えると、システムのパフォーマンスと堅牢性が向上する

また、静的に型付けされることで、想定外の値が紛れ込む心配をしなくても良くなり、プログラムが非常に単純化される

### Juliaの型システムの特徴
- オブジェクトか非オブジェクトかという値の区別がない
    - Juliaでは、すべての値は型を持つ真のオブジェクト
    - Juliaの型は、すべてのノードが型として等しく第一級である、完全に連結した単一の型のグラフに属している
- 値のとる型はただ一つであり、実行時に実際にとるものだけである
    - これはオブジェクト指向言語では「実行時型」と呼ばる
    - オブジェクト指向言語において多相型の静的コンパイルを行うときは、この型の違いは重要になる
- 変数ではなく、値だけが型を持つ
    - 変数は値に束縛された単なる名前である
- 抽象型と具象型は両方とも、他の型によるパラメータ化が可能
    - 型以外にも、シンボル、値でその型が `isbits()` で真となるもの、及びこれらのタプルなどによってパラメータ化が可能
    - 参照や制限をする必要がない場合は、型パラメータは省略することができる

### 型注釈
以下のような形式で型注釈を行うことができる

```julia
<式, 変数> :: <型名>
```

型注釈を行うことで以下のようなメリットがある

1. 型注釈によりコードが自己説明的になり、プログラムに想定される動作を確認できるようになる
2. コンパイラが追加的な型情報を利用できるようなり、パフォーマンスが向上することがある

In [1]:
# 整数型の値を宣言
(1 + 2) :: Int

3

In [2]:
# 誤った型を宣言すると TypeError例外が投げられる
(1 + 2) :: AbstractFloat

TypeError: TypeError: in typeassert, expected AbstractFloat, got Int64

In [3]:
# 型注釈を行うと、その型以外の値を代入することはできなくなる
# -> 想定外の値が代入されることで発生するバグを避けることができる

add(x::Int, y::Int) :: Int = x + y

# 100 + 23 -> 123
println(add(100, 23))

# 整数以外の値を渡すとErrorになる
println(add(3.14, 0.86))

123


MethodError: MethodError: no method matching add(::Float64, ::Float64)

### 抽象型
抽象型とは、その子孫となる具象型の集合である

例えば `AbstractFloat` という抽象型は `Float16`, `Float32`, `Float64` という具象型の集合である

これは以下のような型の階層を表現している

```
     AbstractFloat      ... AbstractType
          |
   ----------------
   |      |       |
Float16 Float32 Float64 ... PrimitiveType
```

#### 抽象型の宣言
抽象型は以下のような形式で宣言される

```julia
# 親タイプのない抽象型
abstract type 型名 end

# 親タイプのある抽象型
abstract type 型名 <: 親型名 end
```

Juliaの型システムにおいて、階層の最上位は `Any`型である

そのため、親タイプの指定がない場合、その型は `Any`型の直下の子タイプになる

逆に、階層の最下位は `Union{}`型であり、全ての型は `Union{}`型の親タイプとなる

例えば、Juliaにおける数値型の階層は以下のようになっている

```
              Any
               |
             Number
               |
              Real
               |
       -----------------
       |               |
    Integer            |
       |               |
   ---------           |
   |       |           |
Signed  Unsigned  AbstractFloat
   |       |           |
Union{}  Union{}    Union{}
```

数値型は以下のように定義されている

```julia
abstract type Number end
abstract type Real          <: Number  end
abstract type AbstractFloat <: Real    end
abstract type Integer       <: Real    end
abstract type Signed        <: Integer end
abstract type Unsigned      <: Integer end
```

In [4]:
# <: 演算子は「is a instance of」（〜の子タイプである）を意味する

## Integer は Number の子タイプか -> true
Integer <: Number

true

In [5]:
# Integer は AbstractFloat の子タイプか -> false
Integer <: AbstractFloat

false

### 原始型（プリミティブ型）
原始型は、データが普通のビットで構成される具象型である

典型的な原始型は、整数や浮動小数点である

#### 原始型の宣言
Juliaでは、以下のような形式で原始型を独自に宣言することが可能である

```julia
# 親タイプのない原始型（Any型の子タイプとなる）
primitive type 型名 ビット数 end

# 親タイプのある原始型
primitive type 型名 <: 親型名 ビット数 end
```

また、Juliaにおける全ての原始型は、以下のようにJulia自身によって定義されている

```julia
primitive type Float16 <: AbstractFloat 16 end
primitive type Float32 <: AbstractFloat 32 end
primitive type Float64 <: AbstractFloat 64 end

primitive type Bool    <: Integer 8 end
primitive type Char 32 end

primitive type Int8    <: Signed   8   end
primitive type UInt8   <: Unsigned 8   end
primitive type Int16   <: Signed   16  end
primitive type UInt16  <: Unsigned 16  end
primitive type Int32   <: Signed   32  end
primitive type UInt32  <: Unsigned 32  end
primitive type Int64   <: Signed   64  end
primitive type UInt64  <: Unsigned 64  end
primitive type Int128  <: Signed   128 end
primitive type UInt128 <: Unsigned 128 end
```

抽象型、原始型を合わせて数値型の階層構造を見ると、以下のようになっている

```
                              Any
                               |
               ---------------------------------
               |                               |
             Number                            |
               |                               |
              Real                             |
               |                               |
       -----------------------------           |
       |                           |           |
    Integer                        |           |
       |                           |           |
 ------------------                |           |
 |                |                |           |
 |          ------------           |           |
 |          |          |           |           |
 |       Signed      Unsigned  AbstractFloat   |
 |          |          |           |           |
Union{}  Union{}     Union{}    Union{}      Union{}
 |          |_ Int8    |_ UInt8    |           |
 |          |_ Int16   |_ UInt16   |           |
 |          |_ Int32   |_ UInt32   |_ Float16  |
 |          |_ Int64   |_ UInt64   |_ Float32  |
 |_ Bool    |_ Int128  |_ UInt128  |_ Float64  |_ Char
```